# Importing Libraries for Model Training

In [ ]:
import pandas as pd
import csv
import dgl
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from dgllife.model import model_zoo
from dgllife.utils import smiles_to_bigraph
from dgllife.utils import EarlyStopping, Meter
from dgllife.utils import AttentiveFPAtomFeaturizer
from dgllife.utils import AttentiveFPBondFeaturizer
import os
import random
import numpy as np
from dgllife.data import MoleculeCSVDataset

# Determining the Computational Device: GPU or CPU

In [ ]:
if torch.cuda.is_available():
    print('use GPU')
    device = 'cuda'
else:
    print('use CPU')
    device = 'cpu'

# Initializing Random Seeds

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

torch.manual_seed(seed)            
torch.cuda.manual_seed(seed)       
torch.cuda.manual_seed_all(seed)

def set_random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

# Data Import and Processing¶

In [ ]:
def collate_molgraphs(data):
    assert len(data[0]) in [3, 4], \
        'Expect the tuple to be of length 3 or 4, got {:d}'.format(len(data[0]))
    if len(data[0]) == 3:
        smiles, graphs, labels = map(list, zip(*data))
        masks = None
    else:
        smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if masks is None:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)
    return smiles, bg, labels, masks

In [ ]:
atom_featurizer = AttentiveFPAtomFeaturizer(atom_data_field='hv')
bond_featurizer = AttentiveFPBondFeaturizer(bond_data_field='he')
n_feats = atom_featurizer.feat_size('hv')
e_feats = bond_featurizer.feat_size('he')

In [ ]:
dc_listings1 = pd.read_csv('train.csv')
dc_listings2 = pd.read_csv('valid.csv')
dc_listings3 = pd.read_csv('test.csv')

In [ ]:
def load_data(data,name,load):
    dataset = MoleculeCSVDataset(data,
                                 smiles_to_graph=smiles_to_bigraph,
                                 node_featurizer=atom_featurizer,
                                 edge_featurizer= bond_featurizer,
                                 smiles_column='SMILES',
                                 cache_file_path=str(name)+'_dataset.bin',
                                 task_names=['pIC50_PIM1','pIC50_PIM2','pIC50_PIM3'],
                                 load=load,init_mask=True,n_jobs=8
                            )
    return dataset

In [ ]:
train_datasets = load_data(dc_listings1,'train',True)
val_datasets = load_data(dc_listings2,'valid',True)
test_datasets = load_data(dc_listings3,'test',True)
train_loader = DataLoader(train_datasets, batch_size=20,shuffle=True,
                          collate_fn=collate_molgraphs)
vali_loader = DataLoader(val_datasets,batch_size=300,shuffle=True,
                         collate_fn=collate_molgraphs)
test_loader = DataLoader(test_datasets,batch_size=300,shuffle=False,
                         collate_fn=collate_molgraphs)

# Training, Evaluation, and Prediction Functions for the AFP-MTRM

In [ ]:
def run_a_train_epoch(n_epochs, epoch, model, data_loader, loss_criterion, optimizer):
    model.train()
    losses = []
    train_meter = Meter()
    for batch_id, batch_data in enumerate(data_loader):
        batch_data
        smiles, bg, labels, masks = batch_data
        bg=bg.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        n_feats = bg.ndata.pop('hv').to(device)
        e_feats = bg.edata.pop('he').to(device)
        prediction = model(bg, n_feats, e_feats)
        loss = (loss_criterion(prediction, labels) * (masks != 0).float()).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_meter.update(prediction, labels, masks)
        losses.append(loss.data.item())
    total_r2 = np.mean(train_meter.compute_metric('r2'))
    total_loss = np.mean(losses)
    if epoch % 10 == 0:
        print('epoch {:d}/{:d}, training_r2 {:.4f}, training_loss {:.4f}'.format(epoch + 1, n_epochs, total_r2,total_loss))
    return total_r2, total_loss

def run_an_eval_epoch(n_epochs, model, data_loader,loss_criterion):
    model.eval()
    val_losses=[]
    eval_meter = Meter()
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            bg = bg.to(device)
            labels = labels.to(device)
            masks = masks.to(device)
            n_feats = bg.ndata.pop('hv').to(device)
            e_feats = bg.edata.pop('he').to(device)
            vali_prediction = model(bg, n_feats, e_feats)
            val_loss = (loss_criterion(vali_prediction, labels) * (masks != 0).float()).mean()
            val_loss=val_loss.detach().cpu().numpy()
            val_losses.append(val_loss)
            eval_meter.update(vali_prediction, labels, masks)
        total_score = np.mean(eval_meter.compute_metric('r2'))
        total_loss = np.mean(val_losses)
    return total_score, total_loss
def predict(model, data_loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            bg = bg.to(device)
            labels = labels.to(device)
            masks = masks.to(device)
            n_feats = bg.ndata.pop('hv').to(device)
            e_feats = bg.edata.pop('he').to(device)
            pred = model(bg, n_feats, e_feats)
            preds.append(pred)
    return preds

# Model construction

In [ ]:
model = model_zoo.AttentiveFPPredictor(node_feat_size=n_feats,
                                   edge_feat_size=e_feats,
                                   num_layers=2,
                                   num_timesteps=1,
                                   graph_feat_size=300,
                                   n_tasks=3,
                                   dropout=0.35
                                    )
model = model.to(device)

# AFP-MTRM model training and evalutation

In [ ]:
loss_fn = nn.MSELoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.00001)

In [ ]:
best_param ={}
best_param["train_epoch"] = 0
best_param["valid_epoch"] = 0
best_param["train_MSE"] = 9e8
best_param["valid_MSE"] = 9e8
n_epochs = 500
for e in range(n_epochs):
    score, loss = run_a_train_epoch(n_epochs, e, model, train_loader, loss_fn, optimizer)
    val_score, val_loss = run_an_eval_epoch(n_epochs, model, vali_loader,loss_fn)
    if loss < best_param["train_MSE"]:
        best_param["train_epoch"] = e
        best_param["train_MSE"] = loss
    if val_loss < best_param["valid_MSE"]:
        best_param["valid_epoch"] = e
        best_param["valid_MSE"] = val_loss
        if val_loss < 0.6:
             torch.save(model, str(e)+'.pt')
    print("EPOCH:\t"+str(e)+'\n'\
        +"train_MSE"+":"+str(loss)+'\n'\
        +"valid_MSE"+":"+str(val_loss)+'\n'\
         +"train_R2"+":"+str(score)+'\n'\
         +"val_R2"+":"+str(val_score)+'\n'\
        )
    if (e - best_param["train_epoch"] >18) and (e - best_param["valid_epoch"] >35):        
        break

# Predicting Labels of Test Set Data

In [ ]:
model = torch.load(str(best_param["valid_epoch"])+'.pt')   

In [ ]:
preds = predict(model,test_loader)

In [ ]:
with open('testpred.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in preds:
        for value in row:
            csvwriter.writerow([value])